In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from PIL import Image
from io import BytesIO
import pytesseract
import pyautogui
import csv
import os
import time
import re
import pandas as pd
import easyocr
pytesseract.pytesseract.tesseract_cmd = r'C:\Archivos de programa\Tesseract-OCR\tesseract.exe'
path = r"C:\Users\ferna\Desktop\IRONHACK\Web Scraping\your-code\driver\chromedriver.exe"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_extension(r'driver/Chollometro-Info-de-precios-alertas-y-más.crx')
chrome_options.add_argument(r'user-data-dir=C:\Users\ferna\Desktop\IRONHACK\Web Scraping\your-code\cookies') 
chrome_options.add_argument('--start-maximized')


In [ ]:
## USANDO LOS URL SACADOS ANTERIORMENTE SE PRUEBA UN NUEVO METODO DE OCR

In [25]:
product_ids = [64, 65, 68, 70, 86, 98, 8, 1, 2, 4, 3, 103, 104, 102, 101, 100, 96, 95, 94, 93, 92, 91, 90, 89, 88, 85, 84, 83, 82, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 69, 67, 66, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39,38,37,36,35,34,33,32,31,30,29,28,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,7,6,5]

TypeError: object of type 'WebElement' has no len()

In [55]:
## DIRECTO DESDE URL

df = pd.read_csv('urls_mediamarkt')

In [60]:
urls = df['product_url']

num_capturas = 374  

reader = easyocr.Reader(['en'])


product_ids = df['Product ID']

for i in range(len(urls)):
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    driver.get(urls[i])
    
    try:
        extension = driver.find_element(By.CSS_SELECTOR, '#assistant')
        shadow_root = driver.execute_script('return arguments[0].shadowRoot', extension)

        time.sleep(4)
        shadow_root.find_elements(By.CSS_SELECTOR, 'div > div > div > div > div._logoContainer_1rkzr_42._opacity-100_1rkzr_52 > svg > path:nth-child(1)')[0].click()
        time.sleep(2)
        shadow_root.find_elements(By.CSS_SELECTOR, 'div > div > div > div > div._iconContainer_1rkzr_20 > div._container_6oko0_1.productStats > div._icon_6oko0_18 > svg')[0].click()
    except IndexError:
        print("La extensión no fue encontrada. Regresando a la pantalla anterior.")
        driver.back()

    # Vuelve a intentar encontrar la extensión después de retroceder
        try:
            extension = driver.find_element(By.CSS_SELECTOR, '#assistant')
            shadow_root = driver.execute_script('return arguments[0].shadowRoot', extension)

            time.sleep(4)
            shadow_root.find_elements(By.CSS_SELECTOR, 'div > div > div > div > div._logoContainer_1rkzr_42._opacity-100_1rkzr_52 > svg > path:nth-child(1)')[0].click()
            time.sleep(2)
            shadow_root.find_elements(By.CSS_SELECTOR, 'div > div > div > div > div._iconContainer_1rkzr_20 > div._container_6oko0_1.productStats > div._icon_6oko0_18 > svg')[0].click()
        
        except IndexError:
            print("La extensión sigue sin encontrarse. Revisa el flujo del programa.")

    x_inicial = 112
    y_inicial = 495

    with open(f'extracciones_mediamarkt6.csv', 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        product_id = product_ids[i]
        product_url = urls[i]
        shop_id = 1
        # Añadir una columna "Product ID"
        csv_writer.writerow(['Product ID', 'N_Extracción', 'Fecha', 'Precio', 'Shop_id', 'product_url', 'texto_entreno'])  # Encabezado del CSV

        for j in range(num_capturas):
            # Definir las coordenadas de recorte (a, b, width, height)
            a, b, width, height = (110, 370, 750, 48)

            # Mueve el ratón a la nueva posición
            pyautogui.moveTo(x_inicial, y_inicial)
            ##time.sleep(0.5)
            # Toma una captura de pantalla
            screenshot = driver.get_screenshot_as_png()
            image = Image.open(BytesIO(screenshot))

            # Recorta la imagen y guárdala con un número distinto cada vez
            image_cortada = image.crop((a, b, a + width, b + height))
            image_cortada.save(f"ventana_flotante_{i}_{j}.png")

            texto_extraído = reader.readtext(f"ventana_flotante_{i}_{j}.png")

            fecha = "0"
            precio = 0.0

            try:
    # Intentar extraer la fecha
                fecha = texto_extraído[0][1].replace('.', ':')  # Formatear la fecha si es necesario

    # Intentar extraer el precio y formatearlo
                precio_str = texto_extraído[1][1].split(' ')[0]  # Eliminar el símbolo de euro
                precio = precio_str.replace(',', '.')  # Convertir ',' a '.' y redondear a dos decimales

            except IndexError:
    # Manejar la excepción si hay un error de índice
                fecha = "0"
                precio = 0.0
            

            csv_writer.writerow([product_id, f"Extracción {j}", fecha, precio, shop_id, product_url, texto_extraído])

            os.remove(f"ventana_flotante_{i}_{j}.png")

            x_inicial += 1.95


    driver.quit()

# Cierra el navegador al final de todas las repeticiones
driver.quit()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


C:\Users\ferna\AppData\Local\Temp\ipykernel_14428\215627852.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [19]:
url= 'https://www.worten.es/cliente/cuenta#/myWishlist'

driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.get(url)


C:\Users\ferna\AppData\Local\Temp\ipykernel_14428\3928003282.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [45]:
enlaces = driver.find_elements(By.CSS_SELECTOR, 'a.w-product__url[href^="/productos/"]')
urls = [enlace.get_attribute('href') for enlace in enlaces]


In [49]:
product_ids = [1,2,4,3,103,104,102,100,96,95,94,93,92,91,90,89,88,86,85,84,83,82,80,79,78,77,76,75,74,73,72]
len(product_ids)

31

In [ ]:
## TODOS LOS URL DE AMAZON

In [80]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Inicia el navegador (asegúrate de tener el driver correspondiente instalado y en el PATH)

# Encuentra todos los elementos <h2>
h2_elements = driver.find_elements(By.XPATH, '//h2')

# Inicializa una lista para almacenar las URLs encontradas
all_urls = []

# Itera sobre los elementos <h2> y encuentra los elementos <a> dentro de cada uno
for h2_element in h2_elements:
    # Encuentra los elementos <a> dentro del elemento <h2>
    a_elements = h2_element.find_elements(By.XPATH, './/a[@href]')
    
    # Extrae las URLs de los elementos <a> y las agrega a la lista
    urls = [a.get_attribute('href') for a in a_elements]
    all_urls.extend(urls)

# Imprime las URLs encontradas
all_urls
# Cierra el navegador


['https://www.amazon.es/dp/B0CDMXJ7N6/?coliid=I120B7QWX1XSQG&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B0BZQVWYHT/?coliid=I2TH4E7HPLITPG&colid=5M7C23FQFP2M&psc=0&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B0BK94F97M/?coliid=I1SSAV2TNBFDPQ&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B09NMBJSCL/?coliid=I3D4ROYTWMHHOF&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B09XXXZNB5/?coliid=I1L0FS89891ULR&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B086D26RCX/?coliid=I51I013S5RX28&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B086BR2Y7H/?coliid=I1IZ3XXKQC8RX1&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B08F2TKVHN/?coliid=I21046DT19AWC6&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B0865YZWGP/?coliid=I26

In [15]:
product_ids = [30,29,28,27,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,7,6,5]


In [ ]:
## AMAZON

In [25]:
num_capturas = 374  

product_ids = [30,29,28,27,25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,7,6,5]

reader = easyocr.Reader(['en'])

for i in range(len(all_urls)):
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    driver.get(all_urls[i])
    
    x = 80
    y = 925

    time.sleep(7)

    pyautogui.click(x, y)

    time.sleep(2)

    pyautogui.click(x, y)

    x_inicial = 112
    y_inicial = 495

    with open(f'extracciones_amazon2.csv', 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        product_id = product_ids[i]
        product_url = all_urls[i]
        shop_id = 1
        # Añadir una columna "Product ID"
        csv_writer.writerow(['Product ID', 'N_Extracción', 'Fecha', 'Precio', 'Shop_id', 'product_url', 'texto_entreno'])  # Encabezado del CSV

        for j in range(num_capturas):
            # Definir las coordenadas de recorte (a, b, width, height)
            a, b, width, height = (110, 370, 750, 48)

            # Mueve el ratón a la nueva posición
            pyautogui.moveTo(x_inicial, y_inicial)
            
            # Toma una captura de pantalla
            screenshot = driver.get_screenshot_as_png()
            image = Image.open(BytesIO(screenshot))

            # Recorta la imagen y guárdala con un número distinto cada vez
            image_cortada = image.crop((a, b, a + width, b + height))
            image_cortada.save(f"ventana_flotante_{i}_{j}.png")

            texto_extraído = reader.readtext(f"ventana_flotante_{i}_{j}.png")

            fecha = "0"
            precio = 0.0

            try:
    # Intentar extraer la fecha
                fecha = texto_extraído[0][1].replace('.', ':')  # Formatear la fecha si es necesario

    # Intentar extraer el precio y formatearlo
                precio_str = texto_extraído[1][1].split(' ')[0]  # Eliminar el símbolo de euro
                precio = precio_str.replace(',', '.')  # Convertir ',' a '.' y redondear a dos decimales

            except IndexError:
    # Manejar la excepción si hay un error de índice
                fecha = "0"
                precio = 0.0

            csv_writer.writerow([product_id, f"Extracción {j}", fecha, precio, shop_id, product_url, texto_extraído])

            os.remove(f"ventana_flotante_{i}_{j}.png")

            x_inicial += 1.95


    driver.quit()

# Cierra el navegador al final de todas las repeticiones
driver.quit()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


C:\Users\ferna\AppData\Local\Temp\ipykernel_14428\2619780671.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [15]:
a, b, width, height = (110, 370, 750, 46)
image_cortada = image.crop((a, b, a + width, b + height))
image_cortada.save(f"ventana_flotante_292.png")

texto_extraído = pytesseract.image_to_string(image_cortada)


In [31]:
texto_extraído

[]

In [19]:
import easyocr


result = reader.readtext('ventana_flotante_292.png')
                    


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [20]:
result

[([[387, 3], [539, 3], [539, 23], [387, 23]],
  '2023/10/23 18.26.30',
  0.803336917945538),
 ([[387, 25], [453, 25], [453, 45], [387, 45]],
  '879.00 €',
  0.9878565611862324)]

In [29]:
# Resultado proporcionado

# Extraer la fecha y el precio
fecha = result[0][1]  # La fecha se encuentra en la primera tupla en la segunda posición
precio_str = result[1][1]  # El precio se encuentra en la segunda tupla en la segunda posición

# Formatear la fecha
# Si es necesario convertir el formato de '2023/10/23 18.26.30' a '2023/10/23 18:26:30'
fecha = fecha.replace('.', ':')

# Convertir el precio a un formato numérico
precio = float(precio_str.split(' ')[0].replace(',', '.'))  # Eliminar el símbolo de euro y convertir ',' a '.'

# Imprimir los resultados
print("Fecha:", fecha)
print("Precio:", precio)


Fecha: 2023/10/23 18:26:30
Precio: 879.0


In [20]:
url= 'https://www.amazon.es/hz/wishlist/ls?requiresSignIn=1&ref_=nav_AccountFlyout_wl'

driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.get(url)


C:\Users\ferna\AppData\Local\Temp\ipykernel_14428\313595303.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [5]:
enlaces = driver.find_elements(By.XPATH, '//a[@title]')
urls = [enlace.get_attribute('href') for enlace in enlaces]


In [6]:
len(urls)

188

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Inicia el navegador (asegúrate de tener el driver correspondiente instalado y en el PATH)

# Encuentra todos los elementos <h2>
h2_elements = driver.find_elements(By.XPATH, '//h2')

# Inicializa una lista para almacenar las URLs encontradas
all_urls = []

# Itera sobre los elementos <h2> y encuentra los elementos <a> dentro de cada uno
for h2_element in h2_elements:
    # Encuentra los elementos <a> dentro del elemento <h2>
    a_elements = h2_element.find_elements(By.XPATH, './/a[@href]')
    
    # Extrae las URLs de los elementos <a> y las agrega a la lista
    urls = [a.get_attribute('href') for a in a_elements]
    all_urls.extend(urls)

# Imprime las URLs encontradas
all_urls
# Cierra el navegador


['https://www.amazon.es/dp/B0CDMXJ7N6/?coliid=I120B7QWX1XSQG&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B0BZQVWYHT/?coliid=I2TH4E7HPLITPG&colid=5M7C23FQFP2M&psc=0&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B0BK94F97M/?coliid=I1SSAV2TNBFDPQ&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B09NMBJSCL/?coliid=I3D4ROYTWMHHOF&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B09XXXZNB5/?coliid=I1L0FS89891ULR&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B086D26RCX/?coliid=I51I013S5RX28&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B086BR2Y7H/?coliid=I1IZ3XXKQC8RX1&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B08F2TKVHN/?coliid=I21046DT19AWC6&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B0865YZWGP/?coliid=I26

In [10]:
len(all_urls)

88

In [65]:
urls = 

['https://www.amazon.es/dp/B0CDMXJ7N6/?coliid=I120B7QWX1XSQG&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it_im',
 'https://www.amazon.es/dp/B0CDMXJ7N6/?coliid=I120B7QWX1XSQG&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B0BZQVWYHT/?coliid=I2TH4E7HPLITPG&colid=5M7C23FQFP2M&psc=0&ref_=list_c_wl_lv_ov_lig_dp_it_im',
 'https://www.amazon.es/dp/B0BZQVWYHT/?coliid=I2TH4E7HPLITPG&colid=5M7C23FQFP2M&psc=0&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B0BK94F97M/?coliid=I1SSAV2TNBFDPQ&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it_im',
 'https://www.amazon.es/dp/B0BK94F97M/?coliid=I1SSAV2TNBFDPQ&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B09NMBJSCL/?coliid=I3D4ROYTWMHHOF&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it_im',
 'https://www.amazon.es/dp/B09NMBJSCL/?coliid=I3D4ROYTWMHHOF&colid=5M7C23FQFP2M&psc=1&ref_=list_c_wl_lv_ov_lig_dp_it',
 'https://www.amazon.es/dp/B09XXXZNB

In [ ]:
num_capturas = 374  

product_ids = [1,2,4,3,103,104,102,100,96,95,94,93,92,91,90,89,88,87,86,85,84,83,82,81,80,79,78,77,76,75,74,73,72]

reader = easyocr.Reader(['en'])

for i in range(len(urls)):
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    driver.get(urls[i])
    
    x = 80
    y = 925

    time.sleep(4)

    pyautogui.click(x, y)

    time.sleep(2)

    pyautogui.click(x, y)

    x_inicial = 112
    y_inicial = 495

    with open(f'extracciones_worten2.csv', 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        product_id = product_ids[i]
        product_url = urls[i]
        shop_id = 1
        # Añadir una columna "Product ID"
        csv_writer.writerow(['Product ID', 'N_Extracción', 'Fecha', 'Precio', 'Shop_id', 'product_url', 'texto_entreno'])  # Encabezado del CSV

        for j in range(num_capturas):
            # Definir las coordenadas de recorte (a, b, width, height)
            a, b, width, height = (110, 370, 750, 48)

            # Mueve el ratón a la nueva posición
            pyautogui.moveTo(x_inicial, y_inicial)
            
            # Toma una captura de pantalla
            screenshot = driver.get_screenshot_as_png()
            image = Image.open(BytesIO(screenshot))

            # Recorta la imagen y guárdala con un número distinto cada vez
            image_cortada = image.crop((a, b, a + width, b + height))
            image_cortada.save(f"ventana_flotante_{i}_{j}.png")

            texto_extraído = reader.readtext(f"ventana_flotante_{i}_{j}.png")

            fecha = "0"
            precio = 0.0

            try:
    # Intentar extraer la fecha
                fecha = texto_extraído[0][1].replace('.', ':')  # Formatear la fecha si es necesario

    # Intentar extraer el precio y formatearlo
                precio_str = texto_extraído[1][1].split(' ')[0]  # Eliminar el símbolo de euro
                precio = precio_str.replace(',', '.')  # Convertir ',' a '.' y redondear a dos decimales

            except IndexError:
    # Manejar la excepción si hay un error de índice
                fecha = "0"
                precio = 0.0

            csv_writer.writerow([product_id, f"Extracción {j}", fecha, precio, shop_id, product_url, texto_extraído])

            os.remove(f"ventana_flotante_{i}_{j}.png")

            x_inicial += 1.95


    driver.quit()

# Cierra el navegador al final de todas las repeticiones
driver.quit()

In [16]:
df = pd.read_csv('../data/CSV/extracciones_worten1.csv')

In [17]:
df = df[['Product ID', 'N_Extracci�n', 'Fecha', 'Precio', 'Shop_id', 'product_url']]


In [18]:
df.to_csv('extracciones_worten_final_1.csv')